In [2]:
import pandas as pd
import requests
import pytest
import datetime as dt
import json
import sys
import os
import sqlite3 as db

In [3]:
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [4]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [5]:
def get_hotspots():
    trailheadHotspots = pd.DataFrame()
    histHotObs = pd.DataFrame()
    cnx = connectDB()
    try:
        #get the latest hotspots from the database
        trailheadHotspots = pd.read_sql('select * from Hotspots',con=cnx,parse_dates=['latestObsDt','latestUpdate'])
        trailheadHotspots.set_index('index',inplace=True)
        trailheadHotspots.sort_values(by=['locId','latestObsDt'],ascending=False,ignore_index=True,inplace=True)
        uniqueHotspots = trailheadHotspots[['locId','latestObsDt']].drop_duplicates(subset=['locId'],keep='first')
        #get hotspots from the observations in the database
        histHotObs = pd.read_sql('select locId,obsDt from historicObservations',con=cnx,parse_dates=['obsDt'])
        min_obsDt = min(histHotObs['obsDt'])
        histHotObs.sort_values(by=['locId','obsDt'],ascending=False,ignore_index=True,inplace=True)
        histHotObs.drop_duplicates(subset=['locId'],keep='first',inplace=True)
        #merge the tables
        hotspots = pd.merge(left=uniqueHotspots,right=histHotObs,how='left',left_on='locId',right_on='locId')
        #replace null, NaT observation date values with the earliest value found in the table
        hotspots.loc[(hotspots['obsDt'].isna()) | (hotspots['obsDt'].isnull()) | (hotspots['obsDt'] == ''),'obsDt'] = min_obsDt
        #recency check
        for x in hotspots:
            dateDiff = hotspots['latestObsDt']-hotspots['obsDt']
            hotspots['dateDiff'] = dateDiff.dt.days
        hotspots['run'] = hotspots['dateDiff'].between(7,700,inclusive='both')
        hotspots = hotspots[hotspots['run'] == False]
        dfk = hotspots[['locId','latestObsDt','obsDt']]
    except Exception as ee:
        raise UserWarning(ee)
    finally: cnx.close()
    return dfk

Duplicate Check

Duplicate checking is done here rather than in the hotspot factory to prevent shelling of the eBird API while still allowing trailheads in close proximity to each other to pull the same observation data. 

##Checking for new hotspots and observations
1. Compare the latest trailheadHotspots eBird locId's with the eBird locId's in the cooking DB
2. For new locId's:
    1. Fetch historical data from the beginning of 2019 to today()-1 from the eBird REST API
    2. Concat and export the results as a .csv
3. For locId's with observations in the DB, obtain the most recent observation date from the eBird REST API for that hotspot:
    1. Make a list of locId's WHERE eBird's latest observation date is more recent than the latest observation date in the DB. Using the most recent DB obsDt as a start date and the most recent eBird API obsDt as an end date, fetch historical sightings from the eBird service
    2. IF the latest obsDt for a locId/hotspot from eBird is within 15 days of the latest obsDt in the DB, remove the locId from the list of locId's to be passed into the historical eBird sighting service
        - TODO: #20 write 'aging' and 'decayed' testing, error logging

In [33]:
##TODO #22 recency refactor: get the latest date some other way; eBird hotspot info does not contain the latest date
##TODO #23 recency refactor: pare down results of hotspot batches to match known Trailhead Direct eBird hotspots

In [12]:
def get_histObs():
    try:
        dfk_test = pd.DataFrame(get_hotspots().head(2))
        if dfk_test.empty:
            raise Exception('The provided dataframe is empty. There are either no eBird hotspots with new checklists or an error has occurred in the evaluation of the get_hotposts() function.')
        dfk_test
    except Exception as exc:
        raise exc
    return dfk_test

In [13]:
get_histObs()

,locId,latestObsDt,obsDt
0,L9614301,2022-02-04 15:12:00,2019-01-01 08:18:00
1,L854460,2022-01-23 10:16:00,2022-01-23 10:16:00


In [38]:
ObsHist = []
import time

for locId in dfk_test.itertuples():
    for x in pd.date_range(start=locId.obsDt,end=locId.latestObsDt,freq='D'):
        time.sleep(0.3)
        ymd = '{}/{}/{}'.format(x.year,x.month,x.day)
        ebird_baseUrl = 'https://api.ebird.org/v2/data/obs/'
        ebird_url = ebird_baseUrl + locId.locId + '/historic/' + ymd
        ebird_auth = {'X-eBirdApiToken': ebird_token}
        ebird_params = {
            'fmt':'json',
            'detail':'simple'
        }
        ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_auth)
        if ebird_request.status_code == requests.codes.ok:
            ebird_resp = pd.DataFrame(ebird_request.json())
            ObsHist.append(ebird_resp)
        ebird_request.raise_for_status()

In [39]:
##TODO #29 write the new observations to the database --DONE
historicObservations = pd.concat(ObsHist)

In [40]:
historicObservations.to_sql(name='historicObservations_cooking',con=cnx,if_exists='append')
##close the connection
cnx.close()